<a href="https://colab.research.google.com/github/aguilin1/tda_ai_text_generation/blob/main/tda_ai_text_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/aguilin1/tda_ai_text_generation.git

Cloning into 'tda_ai_text_generation'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 27 (delta 13), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (27/27), 110.76 KiB | 852.00 KiB/s, done.
Resolving deltas: 100% (13/13), done.


In [3]:
import csv

DATA_FILE = '/content/tda_ai_text_generation/research-abstracts-labeled-first-100.csv'

human_texts = []
ai_texts = []
with open(DATA_FILE) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    next(csv_reader, None) # Skip header row
    for row in csv_reader:
      if row[1] == '0':
        human_texts.append(row[2])
      else:
        ai_texts.append(row[2])
# Print first 2 rows to sanity check
ai_texts[:2]

['In this study, we investigate the coupling loss on bi-columnar BSCCO/Ag tapes using a.c. susceptibility measurements. The bi-columnar structure of BSCCO tapes is known to offer several advantages over traditional tape configurations, including increased tolerance to magnetic field disturbances. However, the effects of the Bi-2212/Ag interface on the coupling between the superconducting filaments of the BSCCO tape is not well understood. Our experiments show that the coupling loss is dominated by the Bi-2212/Ag interface and varies significantly with the orientation and magnitude of the applied a.c. magnetic field. Specifically, coupling loss is found to be lower for in-plane magnetic fields and higher for out-of-plane magnetic fields. We also observe that the annealing of the tapes significantly affects the coupling loss, as annealed tapes exhibit lower loss values than unannealed tapes. Furthermore, we find that the coupling loss is sensitive to the orientation of the Ag matrix, as 

In [4]:
!pip install nltk

In [5]:
import string
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words("english"))
wnl = WordNetLemmatizer()

text = ai_texts[0]

sentences = sent_tokenize(text)

proceesed_sentences = []
for sentence in sentences:
  # Uncomment to remve punctuation if not doing sentence-level transform
  sentence = sentence.translate(str.maketrans('', '', string.punctuation))

  filtered_text = []

  for word in word_tokenize(sentence):
    if word not in stop_words:
      filtered_text.append(wnl.lemmatize(word, pos="v").lower())

  post_filtered_text = ' '.join(filtered_text)
  proceesed_sentences.append(post_filtered_text)

proceesed_sentences

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


['in study investigate couple loss bicolumnar bsccoag tap use ac susceptibility measurements',
 'the bicolumnar structure bscco tap know offer several advantage traditional tape configurations include increase tolerance magnetic field disturbances',
 'however effect bi2212ag interface couple superconducting filaments bscco tape well understand',
 'our experiment show couple loss dominate bi2212ag interface vary significantly orientation magnitude apply ac magnetic field',
 'specifically couple loss find lower inplane magnetic field higher outofplane magnetic field',
 'we also observe anneal tap significantly affect couple loss anneal tap exhibit lower loss value unannealed tap',
 'furthermore find couple loss sensitive orientation ag matrix demonstrate measurements tap transverse longitudinal matrix orientation',
 'finally use numerical simulations confirm validity experimental result',
 'overall study provide important insights couple loss mechanisms bicolumnar bsccoag tap highly rele

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
embeddings = vectorizer.fit_transform(proceesed_sentences)
print(embeddings.toarray())

[[1 0 0 0 0 0 0 0 0 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
  0 0 0 1 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1
  0 0 0 0 0 0 0 1 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1
  0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1
  1 1 1 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0
  0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
  1 0 0 0 0 0 1 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 1 0 0 0 1 0 1 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 2 0 0 1 0 1 0 0 0 0 0 0 0
  1 0 0 0 0 0 1 1 2 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 1 2 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 2 1 0 0 0 0 0 0 1 0 0 0 0 0

In [14]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import itertools

# Label each sentence starting at 0, 1, 2, ...
# Track distances as ((sentence 1, sentence 2), distance)
distances = []
for pair1_i, pair2_i in itertools.combinations(range(len(sentences)), 2):
    cos_sim = cosine_similarity(embeddings[pair1_i], embeddings[pair2_i])[0][0]
    dist = 2 * np.arccos(cos_sim) / np.pi
    distances.append(((pair1_i, pair2_i), dist))

for distance in distances:
  print("Distance {} from [{}] to [{}]".format(distance[1], sentences[distance[0][0]], sentences[distance[0][1]]))


Distance 0.9130973832611151 from [In this study, we investigate the coupling loss on bi-columnar BSCCO/Ag tapes using a.c. susceptibility measurements.] to [The bi-columnar structure of BSCCO tapes is known to offer several advantages over traditional tape configurations, including increased tolerance to magnetic field disturbances.]
Distance 0.9445191586266647 from [In this study, we investigate the coupling loss on bi-columnar BSCCO/Ag tapes using a.c. susceptibility measurements.] to [However, the effects of the Bi-2212/Ag interface on the coupling between the superconducting filaments of the BSCCO tape is not well understood.]
Distance 0.8610675931406382 from [In this study, we investigate the coupling loss on bi-columnar BSCCO/Ag tapes using a.c. susceptibility measurements.] to [Our experiments show that the coupling loss is dominated by the Bi-2212/Ag interface and varies significantly with the orientation and magnitude of the applied a.c. magnetic field.]
Distance 0.90778976591

In [15]:
# This section generates and reformats the cosine similarity distances between
# each pair of points and generates an array of all possible threshold values
from sklearn.metrics.pairwise import cosine_similarity
import itertools
import numpy as np

n = len(embeddings.toarray()) # Number of data points
cosSimDistances = np.zeros((n, n)) # preallocate pairwise distance matrix

# Label each sentence starting at 0, 1, 2, ...
# Track distance between sentence i and sentence j in (i,j) entry of matrix
# Matrix will be upper triangular
for pair1_i, pair2_i in itertools.combinations(range(len(sentences)), 2):
    cos_sim = cosine_similarity(embeddings[pair1_i], embeddings[pair2_i])[0][0]
    dist = 2 * np.arccos(cos_sim) / np.pi
    cosSimDistances[pair1_i][pair2_i] = dist
#print(cosSimDistances)
thresholds = np.sort(np.unique(cosSimDistances))
#print(thresholds)

In [16]:
## Helpers functions NOT based on publication

# to find all the unique lists in a large list of lists
def union_lists(lists):
    # Convert each list to a tuple and package as a set
    # (which automatically is unique)
    unique_tuples = {tuple(a_list) for a_list in lists}
    # Convert the tuples back to lists
    unique_arrays = [list(t) for t in unique_tuples]
    return unique_arrays

In [17]:
## Helper functions based on publication
# The functions in this section are tailored implementations of algorithms
# found in the following paper:

# Zomorodian, A. (2010). Fast construction of the Vietoris-Rips complex.
# Computers & Graphics, 34(3), 263–271.
# https://doi.org/https://doi.org/10.1016/j.cag.2010.03.007

# refer to: Article Section 4.2
# inputs: both C0 and C1 from complex K, max dimension (3 = tetrahedra)
def RipsComplex(K_C0, K_C1, k):
  nu = [] # this will be expanded to hold all the simplices of all sizes
  for i in range(len(K_C0)): # for each vertex in C0
    # find the other, lower ordered vertices that have an edge connecting them
    # to this vertex
    [i_N, N] = LowerPairVertices(K_C0, K_C1, i)
    # build all the cofaces
    nu = AddCofaces(K_C0, K_C1, k, K_C0[i], i_N, N, nu)
  return nu

# refer to: Article Section 4.2
# inputs: complex K (C0 and C1), max dimension (3 = tetrahedra), set
# containing a vertex, set containing lower points paired with that vector,
# and current built-up V-R complex
def AddCofaces(K_C0, K_C1, k, tau, i_N, N, nu):
  nu = union_lists(nu + [tau])
  if len(tau) < k:  # if dim(tau) >= k (has k+1 or more points)
    for i in range(len(N)):
      sigma = N[i] + tau # element to be added to nu
      [i_M, M] = LowerPairVertices(K_C0, K_C1, i_N[i])
      M = N and M # intersection means triangle
      nu = AddCofaces(K_C0, K_C1, k, sigma, i_M, M, nu)
  return nu

# refer to: Article Section 4.1
# inputs: both C0 and C1 of complex K, index of the vertex for which to find
# lower ordered points that are paired by an edge.
def LowerPairVertices(K_C0, K_C1, i_u):
  # get indices i_N of lower vertices paired with vertex u, which has index i_u
  i_N = np.nonzero(K_C1[0:i_u,i_u])[0]
  # get the set N of lower vertices v paired with vertex u
  N = list(K_C0[i] for i in i_N)
  return [i_N, N]

In [18]:
## Driver
from scipy.special import comb # for comb() (i.e. "nChooseK")

# set max dimension to compute
K_C0 = []
for i in range(n):
  K_C0.append([i])

dims = 2 # 0 - points, 1 - edges, 2 - triangles, 3 - tetrahedra, etc.
nPts = len(K_C0)
# compute the total number of things that will be in the complex at the max
# threshold, based on the selected # of dimensions
totalSizeNu = nPts # the points will always be in the complex

# compute the total # of simplices that will exist at the max threshold
for i in range(dims):
  # add # of ith dimension entries in the complex at max threshold
  totalSizeNu = totalSizeNu + comb(nPts, i+1, exact=True)

# preallocate big boundary matrix and an array to track the index of the
# simplex in the full complex (and row in the boundary matrix)
# corresponding to each column in the boundary matrix. Assume the boundary
# matrix will have half as many columns as the number of simplices due to
# removal of zero columns during each iteration
boundaryMatrix = np.zeros((totalSizeNu, round(totalSizeNu/2)), dtype=bool)
nuColIndexFromBoundColIndex = np.zeros(round(totalSizeNu/2), dtype=np.int32)
w_bd = 0 # zero boundary matrix columns originally filled in
# preallocate last iteration complex and reset to correct data type
nu_prev = [None] * totalSizeNu
# track number of actual items set in nu_prev
L_prev = 0
# preallocate epoch and dimension trackers to -1 for error checking later
# birthEpoch = -1*ones(1,totalSizeNu,'int16')
birthEpoch = -1*np.ones(totalSizeNu, dtype=np.int16)
deathEpoch = -1*np.ones(totalSizeNu,dtype=np.int16)
dimension = -1*np.ones(totalSizeNu,dtype=np.int8)
# preallocate full complex buildup and reset to correct data type
epochOrderedComplex = [None] * totalSizeNu

# iterate through each threshold value, building up birthEpoch, deathEpoch,
# and dimension arrays corresponding to the epochOrderedComplex and
# allHomologies arrays. The reduced boundary matrix will also be an
# output at each iteration
for epoch in range(len(thresholds)):
  # first determine which edges are in the complex based on threshold
  K_C1 = cosSimDistances <= thresholds[epoch] # logical array of "in" edges
  # compute the complex as a huge array of dot-delimited strings
  nu = RipsComplex(K_C0, K_C1, dims)
  print(epoch)
  print(K_C1)
  print(nu)

0
[[ True False False False False False False False False]
 [ True  True False False False False False False False]
 [ True  True  True False False False False False False]
 [ True  True  True  True False False False False False]
 [ True  True  True  True  True False False False False]
 [ True  True  True  True  True  True False False False]
 [ True  True  True  True  True  True  True False False]
 [ True  True  True  True  True  True  True  True False]
 [ True  True  True  True  True  True  True  True  True]]
[[6], [2], [5], [8], [4], [1], [7], [0], [3]]
1
[[ True False False False False False False False  True]
 [ True  True False False False False False False False]
 [ True  True  True False False False False False False]
 [ True  True  True  True False False False False False]
 [ True  True  True  True  True False False False False]
 [ True  True  True  True  True  True False False False]
 [ True  True  True  True  True  True  True False False]
 [ True  True  True  True  True  True

In [19]:
# playground
K_C1 = cosSimDistances <= thresholds[3] # logical array of "in" edges
nu = []
i = 6
[i_N, N] = LowerPairVertices(K_C0, K_C1, i)
nu = AddCofaces(K_C0, K_C1, 2, K_C0[i], i_N, N, nu)
print(nu)
i = 7
[i_N, N] = LowerPairVertices(K_C0, K_C1, i)
nu = AddCofaces(K_C0, K_C1, 2, K_C0[i], i_N, N, nu)
print(nu)
nu = RipsComplex(K_C0, K_C1, 2)
print(nu)
print(len(nu))
print(len(union_lists(nu+nu)))

[[6]]
[[6], [7]]
[[6], [2], [3, 4], [5], [8], [4], [1], [7], [0, 5], [0], [0, 8], [3]]
12
12


In [20]:





# iterate through each threshold value, building up birthEpoch, deathEpoch,
# and dimension arrays corresponding to the epochOrderedComplex and
# allHomologies arrays. The reduced boundary matrix will also be an
# output at each iteration
for epoch = 1:length(thresholds)
    # first determine which edges are in the complex based on threshold
    K.C1 = eucMetrics <= thresholds(epoch) # logical array of "in" edges
    # compute the complex as a huge array of dot-delimited strings
    nu = RipsComplex(K,dims)

    # find new complex items for this epoch
    newInComplex = setdiff(nu, nu_prev)
    L_new = length(newInComplex)
    L_tot = L_prev + L_new
    # order newInComplex by simplex size so that each simplex can possibly be
    # destroyed by something to its right in its own birth epoch if needed
    newInComplex = orderSimplices(newInComplex)
    # update tracking of epoch number for new complex items
    birthEpoch(L_prev+1:L_tot) = epoch
    # add new items to the ordered list of items. These are used for
    # columns and rows in the big boundary matrix and for naming in the
    # persistence diagrams
    epochOrderedComplex(L_prev+1:L_tot) = newInComplex

    # preallocate temp array to convert to boundary matrix using newItems
    temp = false(L_tot,L_new)
    # fill temp in as boundary matrix of newInComplex
    temp = findNewItemsBoundary(epochOrderedComplex, newInComplex, temp,...
        L_tot, L_new)

    # reduce the expanded boundary matrix
    for newCol = 1:L_new
        # put temp column into next open column of boundary matrix
        boundaryMatrix(1:L_tot,w_bd+1) = temp(:,newCol)
        # find the first column with the same lowest 1, if such exists
        [matchCol, lowOneRow] = findFirstConflictColumn(boundaryMatrix,...
            w_bd+1)
        while matchCol
            # add the conflicting/matching column to the check column, mod2
            boundaryMatrix(:,w_bd+1) = mod(boundaryMatrix(:,w_bd+1) +...
                boundaryMatrix(:,matchCol),2)
            # add the element name for the matching column as a linear comb.
            epochOrderedComplex{L_prev+newCol} =...
                sprintf('#s+#s',...
                epochOrderedComplex{nuColIndexFromBoundColIndex(matchCol)},...
                epochOrderedComplex{L_prev+newCol})
            # reset matchCol. Since the column has been modified, the match
            # column will be different or nonexistent
            [matchCol, lowOneRow] = findFirstConflictColumn(boundaryMatrix,...
                w_bd+1)
        end
        if lowOneRow >= 0 # if so, it's a pivot
            # set death epoch for the killed row
            deathEpoch(lowOneRow) = epoch
            # add one to the width of the boundary matrix
            w_bd = w_bd+1
            # add the index map to the index tracker
            nuColIndexFromBoundColIndex(w_bd) = L_prev+newCol
        end
    end

# latest complex is previous complex for next iteration
nu_prev = nu
L_prev = length(nu_prev)
end
toc

SyntaxError: invalid syntax (<ipython-input-20-68e87f1596dd>, line 5)